In [16]:
import pandas as pd
from datetime import datetime
import time
from tqdm.notebook import tqdm

import FinanceDataReader as fdr
from pykrx import stock
import pymysql
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

HOSTNAME = "localhost"
PORT     = 3306
USERNAME = "lovedh"
PASSWORD = "123"
DATABASE = "etfkor"
CHARSET1  = "utf8"     # MySQL에서 사용할 캐릭터셋 이름
CHARSET2  = "utf-8"    # Python에서 사용할 캐릭터셋 이름

# 데이터베이스 포맷 설정
con_str_fmt = "mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}"
con_str = con_str_fmt.format(USERNAME, PASSWORD, HOSTNAME, PORT, DATABASE, CHARSET1)
pymysql.install_as_MySQLdb()

# mysql과 연결
import MySQLdb
engine = create_engine(con_str, encoding=CHARSET2)
conn = engine.connect()
print('...데이터베이스 연결 완료')


# 한국 etf 리스트 저장
etf_list = fdr.EtfListing('KR')

def get_stock_prices(Symbol):
    global today
    try:
        print(Symbol, end=' ')
        df = stock.get_etf_ohlcv_by_date("20020101", today, Symbol)
        print(df)
    except:
        print(Symbol+':데이터 불러오기 실패', end= ' ')

...데이터베이스 연결 완료


In [17]:
etf_list

,Symbol,Name
0,069500,KODEX 200
1,252670,KODEX 200선물인버스2X
2,102110,TIGER 200
3,122630,KODEX 레버리지
4,153130,KODEX 단기채권
...,...,...
448,285010,KBSTAR 200중공업
449,284980,KBSTAR 200금융
450,334700,KBSTAR 팔라듐선물인버스(H)
451,287310,KBSTAR 200경기소비재


In [18]:
etf_list[etf_list['Symbol']=='364960']

,Symbol,Name
45,364960,TIGER KRX BBIG K-뉴딜


In [19]:

#for symbol in etf_list['Symbol']: 
url = "https://finance.naver.com/item/coinfo.nhn?code=069500"
result = requests.get(url)
bs_obj = BeautifulSoup(result.content, "html.parser")


# 가격정보
price = bs_obj.find('dl', {'class': 'blind'}).find_all('dd')
now_price = int(price[3].text.split()[1].replace(',','')) #현재가
ex_price = int(price[4].text.split()[-1].replace(',','')) #전일종가

print(now_price, ex_price)

# 상장주식수
no_today = bs_obj.find("div", {"class": "first"})
num_listed_shares = int(no_today.find_all("em")[1].text.replace(',',''))

# 기초정보 부분
base_info = bs_obj.find("table",{"class":"tbl_type1"})
base_sub_info = base_info.find_all('span')

# 상세정보
base_index = base_sub_info[0].text # 기초지수
type = base_sub_info[1].text.replace(', ','-') #유형
listed_day = base_info.find_all('td')[-1].text #상장일
commission = bs_obj.find("table",{"summary":"펀드보수 정보"}).find('em').text # 수수료

print(now_price)
print(commission)
print(num_listed_shares)
print(type)
print(base_index)
print(listed_day)

31520 31405
31520
0.150%
159550000
국내주식형-대표지수
코스피 200
2002년 10월 14일


In [22]:
print('...주가 데이터 저장 중')
today = datetime.today().strftime('%Y%m%d')
for Symbol in etf_list['Symbol']:
    try:
        print(Symbol, end=' ')
        stock_df = stock.get_etf_ohlcv_by_date("20020101", today, Symbol)
        stock_df.to_sql(name=Symbol, con=conn, if_exists='replace', index=True)
    except:
        print(Symbol+':데이터 불러오기 실패', end= ' ')
print('...주가 데이터 저장 완료')

...주가 데이터 저장 중
069500 252670 102110 

KeyboardInterrupt: 

In [13]:
stock.get_etf_ohlcv_by_date("20020101", "20201023", '069500')

,NAV,시가,고가,저가,종가,거래량,거래대금,기초지수
날짜,,,,,,,,
2002-10-14,7727.89,7800,8000,7590,7750,2577230,19915000000,77.27
2002-10-15,7921.53,7850,7980,7700,7940,2288769,17893000000,79.21
2002-10-16,8010.07,8040,8040,7910,7970,2039150,16269000000,80.10
2002-10-17,8118.38,7980,8180,7870,8080,2091777,16837000000,81.16
2002-10-18,8456.22,8300,8500,8250,8430,1321300,11076000000,84.50
...,...,...,...,...,...,...,...,...
2020-10-19,31352.20,31330,31550,31280,31300,9345020,293454000000,312.59
2020-10-20,31564.20,31165,31520,31050,31520,9166339,287464000000,314.71
2020-10-21,31669.32,31575,31710,31470,31620,8601602,271862000000,315.76
